In [1]:
import requests
from copy import copy


def dfs(category_id,parent_name):
    url = "https://api.itemscout.io/api/category/{0}/subcategories".format(category_id)
    response = requests.get(url = url)
    
    for d in response.json()['data']:

        d_id = d['id']
        d_name = d['name']
        d_is_leaf = d['is_leaf']
                
        if(d_is_leaf):
            leaf_id = copy(d_id)
            leaf_name = copy(d_name)
            leaf_category.append([leaf_id,leaf_name,parent_name])
            
            
        else:
            dfs(d_id,d_name)
    
paths = []
leaf_category = []
first_category = {
                        1:'패션의류',
                        2:'패션잡화',
                        3:'화장품/미용',
                        4:'디지털/가전',
                        5:'가구/인테리어',
                        8:'스포츠/레저',
                        9:'생활/건강'
                    } #출산/육아(6) , 식품(7) 제외


starting_category = [1,2,3,4,5,8,9]

for category in starting_category:
    parent_name = first_category[category]
    dfs(category,parent_name)
#dfs(1)


print(len(leaf_category))
#1 : 패션의류 --> 68개 
#2 : 패션잡화 --> 264개
#3 : 화장품/미용 --> 157개
#4 : 디지털/가전 --> 681개
#5 : 가구/인테리어 --> 354개
#6 : 출산/육아 (X)
#7 : 식품 (X)
#8 : 스포츠/레저 --> 카테고리 : 480개, 키워드 : 154207개 --> 필터 적용 : 20000개
#9 : 생활/건강 --> 카테고리 : 1227개, 키워드 : 465817개 --> 필터 적용 : 61751개

#app 1개당 기본 api 12500개 
#계정 1개당 app 10개, api 125000개

In [2]:
import json
import datetime
import urllib.request
from tqdm import notebook
from joblib import Parallel, delayed
from time import sleep
from config import *
from infsell import *
import openpyxl


def extract_keywords(category,client_auth,name_file):
    category_id = category[0]
    category_name = category[1]
    parent_name = category[2]
    
    url_data = "https://api.itemscout.io/api/category/{0}/data".format(category_id)
    url_brands = "https://api.itemscout.io/api/category/{0}/brands".format(category_id)
    
    while True:	
        try: 
            response_data = requests.post(url_data, json=PAYLOAD)
            break
        except Exception as e:
            print(e)
            pass
   
    response_brands = requests.get(url_brands)
    data = response_data.json()['data']['data']
    brands = response_brands.json()['data']
    brands = remove_a_word(brands)
    
    keywords,dict_clicks = get_keywords(data)
    keywords_nb = remove_brands(keywords,brands)
    
    for keyword in keywords_nb:
        encText = urllib.parse.quote(keyword)
        url_excluding_oversea = "https://openapi.naver.com/v1/search/shop.json?display=80&exclude=cbshop&query=" + encText
        url_items = "https://openapi.naver.com/v1/search/shop.json?display=80&query=" + encText # JSON 결과
        
        
        try:
            items_excluding_oversea = get_items(url_excluding_oversea,client_auth)
            items = get_items(url_items,client_auth)
            dict_excluding_oversea = {}
            
            for item in items_excluding_oversea:
                dict_excluding_oversea[item['productId']]=1

            cnt = 0 
            for item in items :
                if item['productId'] not in dict_excluding_oversea:
                    cnt+=1

            info_keyword={}
            info_keyword['카테고리'] = parent_name +' '+ category_name
            info_keyword['키워드'] = keyword
            info_keyword['검색수'] = dict_clicks[keyword]
            info_keyword['해외배송개수'] = cnt

            final_keywords.append(info_keyword)
            
        except Exception as e:
            print('error ',e, ' 키워드 : ', keyword)
            pass
    
            
       

def devide_list(my_list, num):
    return [my_list[i:i+num] for i in range(0,len(my_list),num)]

list_devided = devide_list(leaf_category,160)

#멀티코어 개수
num_core = 16

cnt_keyword=0
path_auth='auth_api.txt'
client_auth = get_auth_api(path_auth) 

date = datetime.datetime.now().strftime('%Y_%m_%d')
name_file = f'{date}_키워드'

for i in notebook.tqdm(range(len(list_devided))):
    final_keywords = []
    with Parallel(n_jobs=num_core, require='sharedmem') as parallel:
        parallel(delayed(extract_keywords)(category,client_auth,name_file)for category in notebook.tqdm(list_devided[i], leave=False))
    
    convert_dict_to_excel(name_file, final_keywords)
        
    

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

...

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/117 [00:00<?, ?it/s]

In [4]:
print(len(final_keywords))

22652
